# Lab 1: Data Exploration & Processing

**Goal:**

   In this lab, we'll explore the pre-processed data to understand the data, determine which features may be impacting to the ML question we are trying to ask so we can ultimately create training/validation/test datasets that will be used for training and evaluation.  
   
   This process is typically iterative in model development so in a real-world scenario you would typically try multiple experiments until you are able to best meet your objective metric.  
      
   * **ML Question**: Is transaction *n* a recurring payment such as a subscription or membership?
   * **Lab Outcome:**: The outcome of this lab is to create a training dataset that we will then use in Lab2

---

**Dataset Information:**
   
   1. This workshop is based on the open public dataset below containing purchase card information from State of Oklahoma: 
     * https://catalog.data.gov/dataset?groups=local&tags=pcard&organization=state-of-oklahoma
     * Dataset: Purchase Card (PCard) Fiscal Year 2015
     * Description:  "*This dataset contains information on purchases made through the purchase card programs administered by the state and higher ed institutions. The purchase card information will be updated monthly after the end of the month.*" 
     
     
   2. The dataset above has been pre-processed to:
   
    * Include MCC Labels (numeric IDs) mapping to the Merchant Category Code (MCC) in the original dataset.  The MCC Labels were pulled using the following public repository: https://github.com/greggles/mcc-codes
    

**Features on Input**: 
   
 Base Features (Part of original dataset):
   * **Year-Month:** yyyymm (Example: 201407)
   * **Agency Num:** Unique Agency Number (Payee) 
   * **Agency Name:** Name of agency (Payee)
   * **Cardholder Last Name:** Last Name of Individual Card Holder
   * **Cardholder First Name:** First Initial of First Name of Individual Card Holder
   * **Description:** Text - Purchase Description (Variable)
   * **Amount:** Amount of purchase
   * **Vendor:** Payee Name
   * **Transaction Date:** Date of purchase in *mm/dd/yy hh:ss* format
   * **Posted Date:** Date transaction posted in *mm/dd/yy hh:ss* format
   * **Merchant Category Code (MCC):** Description of Merchant Category Code
   
Additional Features Added: 
 
   * **MCC_ID:** Numeric representation of Merchant Category Code (MCC)
   * **Recurring_Label:** Label indicating a recurring payment

# Step 1: Load Dataset

In [ ]:
import pandas as pd

#data that was cloned from the github repository ~ loaded into storage on this notebook under /data
input_data = './data/PCARD-workshop-raw.csv'

#read data into a pandas dataframe
df = pd.read_csv(input_data)

#view the first n rows
df.head(10)

Let's pull in the libraries we will use for this lab ...

In [ ]:
import numpy as np                                # For matrix operations and numerical processing
import pandas as pd                               # For munging tabular data
import matplotlib.pyplot as plt                   # For charts and visualizations
import seaborn as sns                             # For charts and visualizations
from IPython.display import Image                 # For displaying images in the notebook
from IPython.display import display               # For displaying outputs in the notebook
from time import gmtime, strftime                 # For labeling SageMaker models, endpoints, etc.
import sys                                        # For writing outputs to notebook
import math                                       # For ceiling function
import json                                       # For parsing hosting outputs
import os                                         # For manipulating filepath names
import sagemaker                                  # Amazon SageMaker's Python SDK provides many helper functions
from sagemaker.predictor import csv_serializer    # Converts strings for HTTP POST requests on inference

---
# Step 2: Explore Data & Feature Processing

In this step, we'll explore the data to understand and shape the data that we'll utilize for training in the next lab. 

### Labels

In the lab, we will be utilizing [Amazon SageMaker: XGBoost Built-In Algorithm](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html).   XGBoost is a supervised learning algorithm so our data must be labeled.  The data has been pre-labeled using the following indicators of a recurring payment for labeling:
     
   1) One of the following, in combination with #2: 
   
   * **Description**: Contains any of the following text in the description:
         -   contains ‘recurr’,'subscript', or 'member' anywhere inside text 
         -   contains ‘monthly’ inside text 
     
   * **MCC_ID**: MCC Code in alignment with recurring payment
         - 5968 : CONTINUITY/SUBSCRIPTION MERCHANTS
      
     **~AND~**
      

   2) Monthly payment (per Cardholder ID) of the same amount (+/- $5) for 3 or more months on same day (+/- 5 days) to same Vendor
             


*Note: The method for labeling above is not prescriptive in how it should be done but providing the base logic for how this dataset was labeled based on a set of assumptions made. Also, our dataset includes only a single year so recurring payments that happen on a yearly basis may perform poorly as they are not labeled in the dataset.*

--- 

#### Label: Recurring_Label

Let's view the distribution of our labeled data...

In [ ]:
df["Recurring_Label"].value_counts()

As we would anticipate, the majority of transactions are not labeled as recurring payments. XGBoost is effective on datasets even when the label is distribution is skewed.  There are methods for handling class imbalance but for now we will continue with our first experiment as a baseline.  We can also see that there are ~211,080 total records.

### Variable Analysis

**Feature Distribution:**

Let's first take a look at how our features are distributed...

In [ ]:
# Frequency tables for each categorical feature

for column in df.select_dtypes(include=['object']).columns:
    display(pd.crosstab(index=df[column], columns='% observations', normalize='columns'))

**Categorical Feature Observations** 

  * **Agency Name:** There are 115 different agencies on input
  * **Description:** The description field is highly variable with 46,888 different descriptions. Note: This field was also used as a basis for a portion of our labeling logic
  * **Vendor:** Vendor is also highly variable. with 47,877 different descriptions

In [ ]:
# Histograms for each numeric features
display(df.describe())
%matplotlib inline
hist = df.hist(bins=30, sharey=True, figsize=(10, 10))

**Numeric Feature Observations** 

  * **Posted_Date_yy & Transaction_Date_yy:** All records have the same value (2014) which would be expected given the dataset captures a specific year. In the current model context, this feature is unlikely to add predictive value. 
  * **MCC_ID:** Although this feature is numeric, it is an ID that corresponds to a categorical feature called 'Merchant Category Code (MCC)' so the MCC_ID feature is really a numeric representation of Merchant Category Code (MCC) using integer encoding.
  
  * **Posted_Date_mm & Transaction_Date_mm:** The input data only contains data from July to December. In this case we are trying to predict recurring payments based on historical data. However, if we were trying to predict whether a current transaction was a recurring payment, we would need to assess how well our model could generalize in January. 

Based on the data and model we are building, let's now drop the features that don't contribute to the variable we are trying to predict.

In [ ]:
model_data = df.drop(['Posted_Date_yy', 'Transaction_Date_yy'], axis=1)
model_data.head(5)

**Feature Correlation:**
    
Let's now take a look at how our features relate to our target variable (recurring)  that we are trying to predict...

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(10, 10))
sns.heatmap(model_data.corr(),annot=True,linecolor='white',linewidths=1)

**Key Observations**: 


1. Highly Correlated Features: 
    - Year-Month & Posted_Date_mm & Transaction_Date_mm:  Because these features have a high correlation, there is little value in having all three features on input.  If we had a multi-year dataset, the year-month or the _yy features that were removed above would potentially have more value.  In this case, we will keep one of the _mm (Transaction_Date_mm) features and drop the others.  
    - Posted_Date_dd & Transaction_Date_dd: Because these features have a high correlation and there is not expected to be a impact on defining a recurring subscription in looking at the posted date vs transaction date, we will drop Posted_Date_dd

In [ ]:
model_data = model_data.drop(['Posted_Date_mm', 'Year-Month','Posted_Date_dd'], axis=1)
model_data.head(5)

### Process categorical feature(s) 

In [ ]:
# Verify categorical features
model_data.info()

**Drop categorical features unlikely to contribute too the predictive value**

   - **Vendor and Description**:  As we discovered in our data exploration above, vendor and description were highly variable.  We also utilized fuzzy matches during our labeling from the Description where the text could be indicative of a recurring payment. 
   - **Agency Name**: The agency name is a categorical representation of the agency number so we will drop the unnecessary categorical feature.
   - **Merchant Category Code (MCC)**: This is a categorical representation of the MCC_ID 

In [ ]:
model_data = model_data.drop(['Vendor', 'Description','Agency Name','Merchant Category Code (MCC)'], axis=1)
model_data.head(10)

The only categorical feature we have left is our label 'Recurring_Label'.  XGBoost requires a labeled dataset so we must convert our labels from 'Yes' or 'No' to '1' or '0' to put it a  format XGBoost will understand for the objective metric we define.  There are many methods to convert our categorical features to numeric. For our purposes, we are going to use [pandas.get_dummies](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) for one-hot encoding. 

The expectected result:

*Original*

| Each Transaction | Recurring_Label |    
| -------- | -------------- |
| 1  |    Yes            |
| 2 | No   | 
| 3 | No |
| 4 | Yes | 

*After one-hot encoding* 

| Each Transaction | Recurring_Label_No |  Recurring_Label_Yes |   
| -------- | -------------- | ------- |
| 1  |    0            |   1
| 2 | 1   |   0 |
| 3 | 1 |  0 |
| 4 | 0 | 1 | 

The conversion above will happen in-place and will not impact our other features (Ex. Cardholder ID, Amount, etc).  We will then be able to use one of the columns created above on training as our label. 

In [ ]:
# Convert our label Subscription_Recurring to numeric (one-hot encoding)
model_data = pd.get_dummies(model_data)
model_data.head(5)

---

# Step 3: Write our training dataset to S3 

In this step, we'll:
   * Split our labeled dataset up into train, validation, and test datasets 
   * Upload train and validation datasets to S3 
   
For simplicity in the lab environment, we are going to upload to the default Amazon S3 bucket used by the SageMaker session.


**Split Data**

When building a model whose primary goal is to predict a target value on new data, it is important to understand overfitting. Supervised learning models are designed to minimize error between their predictions of the target value and actuals, in the data they are given. This last part is key, as frequently in their quest for greater accuracy, machine learning models bias themselves toward picking up on minor idiosyncrasies within the data they are shown. These idiosyncrasies then don't repeat themselves in subsequent data, meaning those predictions can actually be made less accurate, at the expense of more accurate predictions in the training phase.

The most common way of preventing this is to build models with the concept that a model shouldn't only be judged on its fit to the data it was trained on, but also on "new" data. There are several different ways of operationalizing this, holdout validation, cross-validation, leave-one-out validation, etc. For our purposes, we'll simply randomly split the data into 3 uneven groups. The model will be trained on 70% of data, it will then be evaluated on 20% of data to give us an estimate of the accuracy we hope to have on "new" data, and 10% will be held back as a final testing dataset which will be used later on.

| Train | Validation | Test |
| ------ | ------- | ------| 
| <..............70%.............> |  <....20%....> |  <..10%..>  |

In [ ]:
train_data, validation_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data)), int(0.9 * len(model_data))])   # Randomly sort the data then split out first 70%, second 20%, and last 10%

In [ ]:
pd.concat([train_data['Recurring_Label_Yes'], train_data.drop(['Recurring_Label_No', 'Recurring_Label_Yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)

In [ ]:
pd.concat([validation_data['Recurring_Label_Yes'], validation_data.drop(['Recurring_Label_No', 'Recurring_Label_Yes'], axis=1)], axis=1).to_csv('validation.csv', index=False, header=False)

### Upload Training/Validation Data in S3 

The key artifact created from this lab is our training, validation and test datasets that we will use for training and validating our model. XGBoost accepts both a train and optionally a validation dataset on input for training and validation.  We need to ensure our data is loaded to an expected data source for SageMaker training so we will load our data to S3.  

Keep in mind, each algorithm has different options for data input channels. This [quick reference](https://github.com/awsdocs/amazon-sagemaker-developer-guide/blob/master/doc_source/sagemaker-algo-docker-registry-paths.md) provides guidance on common parameters for built-in-algorithms. 

In [ ]:
%%time

import os
import boto3
import re
import sagemaker

role = sagemaker.get_execution_role()
region = boto3.Session().region_name

# S3 bucket for saving code and model artifacts.
bucket = sagemaker.Session().default_bucket()
data_prefix = 'workshop/data'

# bucket path to upload our datasets to
bucket_path = 'https://s3-{}.amazonaws.com/{}'.format(region, bucket)

In [ ]:
# Upload to S3 
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(data_prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(data_prefix, 'validation/validation.csv')).upload_file('validation.csv')

### View Data Uploaded to S3 

Once the training job is complete, the model will be output to an S3 bucket.  This model artifact will be used for hosting our model and getting predictions.  

In [ ]:
from IPython.core.display import Markdown

s3_data = 'https://s3.console.aws.amazon.com/s3/buckets/'+ bucket + '/' + data_prefix + '/?region=us-east-1&tab=overview'
display(Markdown('S3 Data for Training/Evaluation: [link]('+s3_data+')'))

---

# Step 4: Dataprocessing options *(Optional)*

In this notebook and during experimentation on smaller datasets, it's common to do minor pre-processing within a notebook.   However, when processing large datasets at scale or creating a operational process that includes data processing for training/inference it's often not practical to continue to perform data processing in a notebook because of the following reasons: 
   
   * **Cost Optimization:** Performing large data processing on the notebook instance directly can require that you utilize a large notebook instance type than needed.  Because notebook instances generally stay up for longer periods of time, you're paying for unnecessary compute/memory during the time you are processing data.   As an example, the simple data preprocessing done in this notebook on ~33MB of data to prepare our training dataset is not possible on a ml.t2.medium instance due to memory errors.  As a result, we utilize a larger notebook size (ml.t3.large) to be able to process data in the notebook instance
   
   
   * **Operational Effectiveness & Scalability:** Deploying a model to production requires the data for inference to be in a specific format based on the training of that model.  It's not scalable to have all of the data preprocessing logic contained in a notebook that can't be integrated with other systems and solutons.   In addition, an operationally efficient retraining strategy should automate as much as possible including formatting the data for training. 
   
   
   * **Speed:** Speed up processing time by utilizing larger compute/memory 


## SageMaker Processing 

AWS offers several options for effective data cleansing and pre-processing for data science workloads.  One of those includes  [Amazon SageMaker Processing](https://aws.amazon.com/blogs/aws/amazon-sagemaker-processing-fully-managed-data-processing-and-model-evaluation/).  SageMaker processing offers fully managed infrastructure for data processing and model evaluation. Amazon SageMaker Processing includes a Python SDK that uses SageMaker's built-in containerr for scikit-learn as well as the capability to use your own Docker images without having  to conform to any Docker image specification.  

Using the same dataset we imported in Step 1, let's demonstrate how we can take advantage of secondary compute for processing tasks with SageMaker Processing. Much of the preprocessing was done in advance to simplify the workshop and most of what we did above was drop features as well as convert categorical features to numeric.  

Let's read the original dataset into a DataFrame to review original contents...

In [ ]:
#read data into a pandas dataframe
dfp = pd.read_csv(input_data)

#view the first n rows
dfp.head(2)

Upload raw data to S3 ~ where SageMaker Processing will pick it up

In [ ]:
s3_path_to_data = sagemaker.Session().upload_data(bucket=bucket, 
                                                  path='./data/PCARD-workshop-raw.csv', 
                                                  key_prefix='processing-jobs')
print('Print S3 Path:', s3_path_to_data)

Let's review the training dataset to view the format we want to enforce through our processing job...

To achieve this, we need to write a script that can perform the same transformations we executed directly from our notebook instance in Step 3.  

In [ ]:
!head train.csv

Notice we have our label in the first column for XGBoost training

### CHALLENGE #1

Using what you know about the data processing we previously performed to convert the raw data into the final training dataset, write the training pre-processing script that we will then utilize to run a SageMaker Processing job to convert our raw dataset into our training dataset.  

*HINT: The following [SageMaker Notebook Example](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker_processing/scikit_learn_data_processing_and_model_evaluation/scikit_learn_data_processing_and_model_evaluation.ipynb) provides an example for running preprocessing scripts inside processing jobs using the scikit-image. Keep in mind the script is only an example. The one we create below  should mimic the transformations specific to our use case including: 
          -  Dropping non-impacting features
          -  Converting categorical data to numeric 
 

In [ ]:
#%%writefile preprocessing.py

# [Enter preprocessing code here - and convert the cell to code block ]

Let's now run the script we created above as a SageMaker Processing job...

**CHALLENGE HELP:** If you are stuck on the preprocessing script above, comment out the line below and execute the cell to create your preprocessing.py script automatically by pulling in a pre-created version that exists in the /scripts folder of our notebook instance.   To uncomment simply remove the '#' and Run cell. 

In [ ]:
# If you want to use an existing script instead of creating your own - remove the '#' from  the line below and execute
#!cp ./scripts/preprocessing.py .

### SageMaker Processing Setup

To run the scikit-learn preprocessing script as a processing job, create a SKLearnProcessor, which lets you run scripts inside of processing jobs using the scikit-learn image provided. Notice you get to select the instance type and size allowing you to perform processing/cleansing on a much larger instance size that is automatically provisioned and shutdown once processing completes - keeping your notebook size small. 

In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.sklearn.processing import SKLearnProcessor

region = boto3.session.Session().region_name

role = get_execution_role()
sklearn_processor = SKLearnProcessor(framework_version='0.20.0',
                                     role=role,
                                     instance_type='ml.m5.xlarge',
                                     instance_count=1)

Run the preprocessing.py script created above as a processing job using the SKLearnProcessor.run() method. 


Input(s):

   * Processing script (custom code)
   * PCard raw dataset in Amazon S3 as *ProcessingInput* which is read by the script from /opt/ml/processing/input. 
   
   
Output(s): 
   * Source - path the script writes output data to.  For outputs, the destination defaults to an S3 bucket that the Amazon SageMaker Python SDK creates for you, following the format *s3://sagemaker-<region>-<account_id>/<processing_job_name>/output/<output_name/*.
    
   * Output_name - values to make it easier to retrieve these output artifacts after the job is run
    

These local paths inside the processing container must begin with /opt/ml/processing/.

In [ ]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

sklearn_processor.run(code='preprocessing.py',
                      inputs=[ProcessingInput(
                        source=s3_path_to_data,
                        destination='/opt/ml/processing/input')],
                      outputs=[ProcessingOutput(output_name='train_data',
                                                source='/opt/ml/processing/train'),
                               ProcessingOutput(output_name='validation_data',
                                                source='/opt/ml/processing/validation')]
                     )

preprocessing_job_description = sklearn_processor.jobs[-1].describe()

output_config = preprocessing_job_description['ProcessingOutputConfig']
for output in output_config['Outputs']:
    if output['OutputName'] == 'train_data':
        preprocessed_training_data = output['S3Output']['S3Uri']
    if output['OutputName'] == 'validation_data':
        preprocessed_validation_data = output['S3Output']['S3Uri']

Now inspect the output of the pre-processing job, which consists of the processed features. The goal is to get the data in the same format as the training data above. 

In [ ]:
training_features = pd.read_csv(preprocessed_training_data + '/trainp.csv', index_col=None)
print('Training features shape: {}'.format(training_features.shape))
training_features.head(5)

# Congratuations - You've completed Lab 1 

In this lab we explored our dataset, identified features for training and converted categorical features numeric.  We walked through the example of creating our training/validation dataset utilizing the notebook instance compute/memory as well as how you can utilize SageMaker Processing for data cleansing/pre-processing.

We only needed one training dataset for our next lab so will only utilize one of the datasets created above.  The purpose of creating two was purely to demonstrate options for data cleansing/processing.

In [ ]:
# Let's collect & store variables we will need to use for Lab2
# **NOTE: This step is only needed for the nature of our lab setup/environment

%store test_data
%store data_prefix